In [ ]:
import numpy as np

train_path = "./data/train.csv"
test_path = "./data/test.csv"
test_labels = "./data/test_labels.csv"

def load_data(train_path, test_path, test_labels):
    train = pd.read_csv(link + "train.csv")
    test = pd.read_csv(link + "test.csv")
    cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']
    return train, test, cols_target
